<a href="https://colab.research.google.com/github/SmirnygaTotoshka/FBB_Courses/blob/main/ML_Biology_Sem7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Упражнение 1

В данном упраднении на датасете Breast cancer wisconsin следует продемонстрировать работу моделей градиентного бустинга, xgboost, lightgbm и catboost.

Импортируйте необходимые библиотеки указанные в лекции.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Загрузите датасет. Разделите датасет на тестовую и обучающую выборки

In [ ]:
cancer = datasets.load_breast_cancer()
X, y = cancer.data, cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

Определите параметры

In [ ]:
params = {'n_estimators': 100,
          'max_depth': 2,
          'min_samples_split': 6,
          'learning_rate': 0.01,
          'loss': 'ls'}

Обучите модель градиентного бустинга

In [ ]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

Постройте график качества обучения

In [ ]:
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(reg.staged_predict(X_test)):
    test_score[i] = reg.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
##Put your code here

По аналогии обучите модели xgboost, catboost и lightdm

In [ ]:
xg_reg = xgboost.XGBRegressor(learning_rate = 0.1, max_depth = 5, alpha = 10, min_child_weight=9, n_estimators = 10)

lgg_reg = lightgbm.LGBMRegressor(n_estimators=2000, learning_rate=0.1, max_depth=-1, num_leaves=100, min_child_weight=7, n_jobs=-1) 

!pip install catboost

cat_reg = catboost.CatBoostRegressor(iterations=2000, learning_rate=0.1, depth=6, ,min_data_in_leaf=7, verbose=0)

##LEarning process

Оформите результаты ввиде графиков, сделайте выводы

In [ ]:
fig = plt.figure(figsize=(6, 6))
##Put your code here

Определите важность признаков используя Gini impurity и SHAP

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

!pip install shap
import shap

In [ ]:
##Use Gini and Shap

explainer = shap.Explainer(your_reg)
shap_values = explainer(X)

shap.plots.waterfall(shap_values[0])

Отобразите графики (Дополнительно)

In [ ]:
plt.figure(figsize=(8,8))
#Other data

#Упражнение 2

В данном упражнении вам нужно будет оценить важность функциональный групп во взаимодействиях с заданным белком

Загрузите датасет

In [ ]:
!wget https://zenodo.org/record/3362353/files/activity_classes_ChEMBL24.dat?download=1

--2021-10-28 15:07:13--  https://zenodo.org/record/3362353/files/activity_classes_ChEMBL24.dat?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540839 (528K) [application/octet-stream]
Saving to: ‘activity_classes_ChEMBL24.dat?download=1.1’

activity_classes_Ch 100%[===================>] 528.16K  1.04MB/s    in 0.5s    

2021-10-28 15:07:15 (1.04 MB/s) - ‘activity_classes_ChEMBL24.dat?download=1.1’ saved [540839/540839]



In [ ]:
!cat 'activity_classes_ChEMBL24.dat?download=1'

Выходные данные были обрезаны до нескольких последних строк (5000).
CHEMBL233	Mu opioid receptor	CCC(C)NC(=O)N(CCN1C2CCC1CC(C2)c3cccc(c3)C(=O)N)CC4CCCCC4	18511	8
CHEMBL233	Mu opioid receptor	CCC(C)NC(=O)N(CCN1C2CCC1CC(C2)c3cccc(c3)O)CC4CCCCC4	9598	8
CHEMBL233	Mu opioid receptor	CCC(CC)C(=O)N(CCN1C2CCC1CC(C2)c3cccc(c3)O)Cc4ccccc4	10414	8
CHEMBL233	Mu opioid receptor	CCC(CC)C(=O)N(Cc1ccccc1)C(C)CN2C3CCC2CC(C3)c4cccc(c4)O	7001	8
CHEMBL233	Mu opioid receptor	CCC(CC)CN1CCCc2cc(ccc2C1)Oc3ccc(cn3)C(=O)N	23357	7.38
CHEMBL233	Mu opioid receptor	CCC1(C(=O)Nc2ccccc2N(C1=O)C3CCN(CC3)C4CCC(CC4)C(C)C)CC	733	5.3
CHEMBL233	Mu opioid receptor	CCC1(C(=O)Nc2ccccc2N(C1=O)C3CCN(CC3)C4CCCCCCC4)CC	733	5.21
CHEMBL233	Mu opioid receptor	CCC12CCN(CC1Oc3c2cc(cc3)O)CC4CC4	3666	8.4
CHEMBL233	Mu opioid receptor	CCC1C2CCCC3C2C(CCN3CC4CCCCC4)CC1=O	2217	5.72
CHEMBL233	Mu opioid receptor	CCC1C2CCCC3C2C(CCN3Cc4cccs4)CC1=O	2217	6.05
CHEMBL233	Mu opioid receptor	CCC1CC(C(CN1CCc2ccccc2)C)(C)c3cccc(c3)O	405	7.12
CHEMBL233	M

Обработайте датасет и разделите его на тестовую выборку

In [ ]:
activity_classes = []

with open('activity_classes_ChEMBL24.dat?download=1') as rf:
  for l in rf.readlines():
    line = [s.strip() for s in l.split('\t')]
    activity_classes.append(line)
import pandas as pd

df = pd.DataFrame(activity_classes)
df.columns = ['TargetID', 'TargetName', 'SMILES', 'AnalogSeriesID', 'pKi']
df = df.drop(df.index[0])
df

,TargetID,TargetName,SMILES,AnalogSeriesID,pKi
1,CHEMBL233,Mu opioid receptor,C#CCN1CCC23CCCCC2C1Cc4c3cc(cc4)O,1039,11.52
2,CHEMBL233,Mu opioid receptor,C#CCN1CN(C2(C1=O)CCN(CC2)C(c3ccccc3)c4ccccc4)c...,24317,5.37
3,CHEMBL233,Mu opioid receptor,CC(=NC(CO)C(=NCC1CC23CCC1(C4C25CCN(C3Cc6c5c(c(...,23111,8.23
4,CHEMBL233,Mu opioid receptor,CC(=NC(Cc1ccccc1)C(=NCC2CC34CCC2(C5C36CCN(C4Cc...,23111,9.44
5,CHEMBL233,Mu opioid receptor,CC(=O)CN1CN(C2(C1=O)CCN(CC2)C3CCC(c4c3cccc4)(C...,10988,7.85
...,...,...,...,...,...
5293,CHEMBL4792,Orexin receptor 2,c1ccc(c(c1)c2ncccn2)C(=O)N3C4CCC3C(C4)Nc5cnc(c...,6612,6.24
5294,CHEMBL4792,Orexin receptor 2,c1ccc(cc1)C2(CC2C(=O)Nc3ccccn3)COc4ccccc4,4774,6.01
5295,CHEMBL4792,Orexin receptor 2,c1ccc(cc1)CN2C(=O)CCCC23CCN(CC3)c4cnc5ccccc5n4,21610,6.51
5296,CHEMBL4792,Orexin receptor 2,c1ccc(cc1)CN2CCCC3(C2=O)CCN(CC3)c4[nH]c5ccccc5n4,15031,5.97


Установим rdkit, он вам пригодится.

In [ ]:
!pip install rdkit-pypi

In [ ]:
import rdkit
from rdkit import Chem 
from rdkit.Chem import Descriptors

Вытянем из последовательностей смайлз признаки.

In [ ]:
def get_mol_features(s):
  mol = Chem.MolFromSmiles(s)

  tpsa = Descriptors.TPSA(mol)
  logp = Descriptors.MolLogP(mol)
  wt = Descriptors.ExactMolWt(mol)
  ch = Descriptors.MaxPartialCharge(mol)
  rad_e = Descriptors.NumRadicalElectrons(mol)
  val_e = Descriptors.NumValenceElectrons(mol)
  h_acc = Descriptors.NumHAcceptors(mol)
  h_don = Descriptors.NumHDonors(mol)
  ar_r = Descriptors.NumAromaticRings(mol)
  bal = Descriptors.BalabanJ(mol)
  ipc = Descriptors.Ipc(mol)
  nhoh = Descriptors.NHOHCount(mol)
  no = Descriptors.NOCount(mol)
  hka = Descriptors.HallKierAlpha(mol)
  lab = Descriptors.LabuteASA(mol)
  csp = Descriptors.FractionCSP3(mol)
  return tpsa, logp, wt, ch, rad_e, val_e, h_acc, h_don, ar_r, bal, ipc, nhoh, no, hka, lab, csp

In [ ]:
features = []
smiles = df['SMILES']

for s in smiles:
  features.append(get_mol_features(s))

data = pd.DataFrame(features)
data.columns = ['tpsa', 'logp', 'wt', 'ch', 'rad_e', 'val_e', 'h_acc', 'h_don', 'ar_r', 'bal', 'ipc', 'nhoh', 'no', 'hka', 'lab', 'csp']
data['pKi'] = df['pKi'].values.astype(float)
data['smiles'] = df['SMILES'].values

Посмотрите что у вас получилось в data

In [ ]:
from sklearn.preprocessing import StandardScaler

X, y = data.iloc[:, :-2], data['pKi']

X_train, X_test, y_train, y_test = train_test_split#Разделите выборку на обучение и тест

TypeError: ignored

Вфберите две модели, которые вам понравились из первого задания, обучите и проанализиуйте в SHAP

In [ ]:
xg_reg = xgboost.XGBRegressor(learning_rate = 0.1, max_depth = 5, alpha = 10, min_child_weight=9, n_estimators = 10)

lgg_reg = lightgbm.LGBMRegressor(n_estimators=2000, learning_rate=0.1, max_depth=-1, num_leaves=100, min_child_weight=7, n_jobs=-1) 
##Add everything you need, if you need
explainer = shap.Explainer(your_reg)
shap_values = explainer(X)

shap.plots.waterfall(shap_values[0])

Сделайте выводы, есть ли разница между выбранными вами моделями. Какие параметры оказались важными.

Выводы